In [2]:
import re
import pandas as pd
from keras.models import load_model
import numpy as np
import pickle
import json
import os

Using TensorFlow backend.


In [89]:
filename = []
actual_result = []
probability_left = []
probability_right = []
outcome = []

In [90]:

for subdir, dirs, files in os.walk("test_data"):
    for file in files:
        filepath = subdir + os.sep + file
        print()
        
        filename.append(file)
        print(filepath)
        
        if 'clap' in filepath:
            actual_result.append('left')
        else:
            actual_result.append('right')
            
        
        clean_data = cleandata(filepath)
        lines , data = jaw_clench_lines(clean_data)
        final_array = useless_data_removal(data, lines, filepath)

        print("Data has been cleaned!")

        dfObj = pd.DataFrame(final_array, columns = ['Alpha_1','Alpha_2','Alpha_3','Alpha_4','Beta_1','Beta_2','Beta_3','Beta_4','Delta_1','Delta_2','Delta_3','Delta_4','Theta_1','Theta_2','Theta_3','Theta_4','Gamma_1','Gamma_2','Gamma_3','Gamma_4'])
        print('Data has been saved into a dataframe!')
        #print(dfObj)

        model = load_model('model.h5')

        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        classes = model.predict_classes(dfObj, batch_size=5)
        probability = model.predict(dfObj,batch_size=5)




        with open('rf_model.pkl', 'rb') as f:
            rf = pickle.load(f)

            rf_preds = rf.predict(dfObj)


        with open('svm_model.pkl', 'rb') as f:
            svm = pickle.load(f)

            svm_preds = svm.predict(dfObj)


        #print('predictions from Neural Network: ', classes)
        #print('predictions from Random Forest: ', rf_preds)
        #print('predictions from SVM: ', svm_preds)


        left, right, result = results(classes, rf_preds, svm_preds, dfObj)
        
        probability_left.append(left)
        probability_right.append(right)
        outcome.append(result)



test_data/lookright.txt
1871
Data has been cleaned!
Data has been saved into a dataframe!
percentage prediction for Left is  0.1326530612244898
percentage prediction for Right is  0.8673469387755102
The direction predicted is : Right

test_data/clap4.txt
766
Data has been cleaned!
Data has been saved into a dataframe!
percentage prediction for Left is  0.4017857142857143
percentage prediction for Right is  0.5982142857142857

test_data/clap3.txt
815
Data has been cleaned!
Data has been saved into a dataframe!
percentage prediction for Left is  0.453125
percentage prediction for Right is  0.546875

test_data/clap.txt
906
Data has been cleaned!
Data has been saved into a dataframe!
percentage prediction for Left is  0.4666666666666667
percentage prediction for Right is  0.5333333333333333

test_data/clap2.txt
850
Data has been cleaned!
Data has been saved into a dataframe!
percentage prediction for Left is  0.0
percentage prediction for Right is  1.0
The direction predicted is : Right



In [5]:
def cleandata(filepath):
    with open(filepath, 'r') as f2:
                raw_data = f2.read()
                clean_data = raw_data.replace("[8000]", "\n")
                
    return clean_data

In [6]:
def jaw_clench_lines(clean_data):
    
    lines = []
    data = clean_data.split('\n') #split string into a list
    for i,temp in enumerate(data):
        if 'jaw_clench' in temp:
            #print(i)
            lines.append(i)
            
    
    print(len(data))
            
            
    return lines, data

In [7]:
def useless_data_removal(data,lines, filepath):
    
    N = 20
    
    #del data[(lines[2]):]
    del data[:lines[1]]
            
 
            
    gh = "\n".join([str(elem) for elem in data])
    
    data = gh.replace("-∞", "0.00")                 
    data = re.findall("\d+\.\d+", data)
    
   
    subList = [data[n:n+N] for n in range(0, len(data), N)]
    
    return subList

In [87]:

def results(classes, rf_preds , svm_preds, dfObj):
    
    #counts the number of 1s (left predictions)
    left = np.count_nonzero(classes == 1)  + np.count_nonzero(rf_preds == 1) 
    #counts the number of 0s (right predictions)
    right = np.count_nonzero(classes == 0)  + np.count_nonzero(rf_preds == 0)
    length = len(dfObj) #length of the dataset
    accuracy_left = left / (length*2) # average score for left
    accuracy_right = right /(length*2)# average score for right
    
    print("percentage prediction for Left is ", accuracy_left)
    print("percentage prediction for Right is ", accuracy_right)
    
    if accuracy_left > 0.6: #if the accuracy is above 60% then save the direction as L to the json 
        print('The direction predicted is : Left')
        result = 'left'
    elif accuracy_right > 0.6: #if the accuracy is above 60% then save the direction as R to the json
        print('The direction predicted is : Right')
        result = 'right'
    else:
        result = 'No decision'
        
    return accuracy_left , accuracy_right , result


In [91]:
import numpy as np

h = list(zip(filename , probability_left , probability_right, actual_result, outcome))


In [92]:
h

[('lookright.txt', 0.1326530612244898, 0.8673469387755102, 'right', 'right'),
 ('clap4.txt', 0.4017857142857143, 0.5982142857142857, 'left', 'No decision'),
 ('clap3.txt', 0.453125, 0.546875, 'left', 'No decision'),
 ('clap.txt', 0.4666666666666667, 0.5333333333333333, 'left', 'No decision'),
 ('clap2.txt', 0.0, 1.0, 'left', 'right'),
 ('clap1.txt', 0.33783783783783783, 0.6621621621621622, 'left', 'right'),
 ('lookright1.txt',
  0.4915254237288136,
  0.5084745762711864,
  'right',
  'No decision'),
 ('lookright2.txt', 0.0, 1.0, 'right', 'right'),
 ('lookright3.txt',
  0.009615384615384616,
  0.9903846153846154,
  'right',
  'right'),
 ('lookright4.txt', 0.07627118644067797, 0.923728813559322, 'right', 'right')]

In [93]:
from tabulate import tabulate


headers = ["Datasets", "Probability Left", "Probability Right", "Expected_outcome" , "Actual_Outcome"]

# tabulate data
table = tabulate(h, headers, tablefmt="grid")

# output
print(table)

+----------------+--------------------+---------------------+--------------------+------------------+
| Datasets       |   Probability Left |   Probability Right | Expected_outcome   | Actual_Outcome   |
+================+====================+=====================+====================+==================+
| lookright.txt  |         0.132653   |            0.867347 | right              | right            |
+----------------+--------------------+---------------------+--------------------+------------------+
| clap4.txt      |         0.401786   |            0.598214 | left               | No decision      |
+----------------+--------------------+---------------------+--------------------+------------------+
| clap3.txt      |         0.453125   |            0.546875 | left               | No decision      |
+----------------+--------------------+---------------------+--------------------+------------------+
| clap.txt       |         0.466667   |            0.533333 | left               |